### 1. Modelo

Cada modelo:

- representa una clase Python,
- que ser verá reflejada en una o más bases de datos.
- contiene campos y metadata.
- contiene los métodos que establecen el comportamiento de los modelos.
    

#### 1.1 Las  variables de clase de cada modelo representarán los CAMPOS en la base de datos:

In [ ]:
from django.db import models

class Artist(models.Model): #cada modelo hereda de django.db.models.Model
    name = models.Charfield(max_lenght=200) # en este caso declaramos campos de texto
    genre = models.Charfield(max_length=200) # el nombre de cada variable, será el nombre en la base de datos.
    
    # existen distintos tipos de fields para guardar distintos tipos de datos.
    
# este código se incorpora en el archivo models de nuestra app.

#### 1.2 asimismo, existen distintos tipos de relaciones típicas en una base de datos entre fields.
    # por ejemplo:


In [ ]:
class Album(model.Model):
    artist = models.ForeignKey(Artist, on_delete=models.CASCADE) #en este caso la información no enta desde un campo, sino heredando desde otra clase
    name = models.CharField(max_lenght=200)
    pub_date = models.DateField("date published")


### 2. Migraciones

Con el código que acabamos de escribir, django ya tiene la información necesaria para:
    - Generar un esquema de base de datos.
    - Generar una API de acceso a la base de datos a través de objetos de las clases.

Para lo primero, django genera migraciones, que son la **forma en que django guarda los cambios a la base de datos.**

#### 2.1 manage.py migrate

In [ ]:

# mediante el siguiente comando:
python manage.py migrations music
    # python inspecciona nuestros modelos y genera una migración, que al ser ejecutada hará los cambios necesarios en la base de datos.

# EL código Python que correrá la aplicación se encuentra en el archivo migrations de nuestra app. 

#### 2.2 Ejecución del código SQL necesario

In [ ]:
# mediante el siguiente comando:
python manage.py sqlmigrate musica XXX
    # se ejecutará el código SQL necesario, que podemos ver en el siguiente comando, donde XXXX es el número de la migración.

### 3. API de modelos

APIs de servicios web: son las interfaces de desarrollo de aplicaciones que permiten el intercambio de información entre un servicio web (software que da acceso a un servicio concreto a través de una URL) y una aplicación. 

Normalmente ese intercambio se produce a través de peticiones HTTP o HTTPS (la versión cifrada del protocolo HTTP). 

En la petición de la aplicación y respuesta, también en HTTP del servicio web, se contiene información de todo tipo tanto en los metadatos de la cabecera como en los del mensaje, normalmente en dos tipos de formatos muy usados: XML o JSON.

In [ ]:
objects.all()
objects.filter()
objects.get
save()

### 4. Views

- Procesan requests HTTP y retornan HTML, generado desde templates.
- Se escriben en el archivo views.py


In [ ]:
from django.http import HttpResponse

def index(request): #en este caso estoy generando una vista llamada index, la cual toma por argumento un request.
    return HttpResponse("Bienvenidos a la app de música!") # y me devuelve un objeteo (en este caso un response). http de tipo str.

### 5. Views y URLs

- Para ver nuestra view, debemos hacer que alguna URL nos la muestre.

- Para esto, creamos un archivo urls.py en la carpeta de la app correspondiente.

##### El path/ de nuesta app nos mostrará la vista "index" que definimos anteriormente.

In [ ]:
from django.urls import path
from . import views

# este será un arreglo con todas las URLs de nuestra app.
urlpatterns = [path("",views.index, name="index"),] # En este caso, estamos declarando que el path vacío, nos va a mostrar la vista index


##### Incluir la URL de nuestra app al proyecto.

- Lo hacemos en el archivo urls.py del proyecto (donde ya están las URLs del admin)

- En el siguiente arreglo urlpatterns, incluimos todas las rutas que definimos dentro de nuestra app, dentro de la ruta /music del proyecto.

- Esto nos permite incluir nuestras rutas en múltiples proyectos sin que choque con otras.


In [ ]:
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("music/", include("music.urls")), #acá estamos incluyendo las rutas de music.
    path("admin/",admin.site.urls), #estas son las rutas admin
]


##### Volviendo a views:

###### Podemos tener una view que muestre a un artista

In [ ]:
def artist(request, artist_id):
    return HttpResponse("Estás viendo al artista %s.´% artist id") # no está haciendo eco del artist_id

##### Pero esto no está haciendo nada, obtengamos el objeto y mostremos su nombre

In [ ]:
def artist_detail(request, artist_id):
    artist = Artist.objects.get(id=artist_id) # usamos la API de modelos. le pasamos el id del artista que recibimos
    return HttpResponse("Estás viendo al artista %s.´% artist.name")

##### De esta forma, cada vez que modifiquemos los parámetros de la función, se modificará el path de la URL correspondiente, trayendo los datos desde la bdd.

### 6. Templates

 En la clase anterior, generamos documentos html en forma de response, pero no controlamos cómo estos eran generados.


### 6.1 Sistema de templating

- Django tiene un sistma de templating que genera HTML a partirde una mezcla de HTML y código python.

- Podemos renderear estos templates con parámetros que pasamos desde las views.

### Cómo?

1. Creamos un directorio "/templates" dentro de nuestra app.
2. Cremos un directorio dentro del anterior, donde estarán nuestros templates.

### Luego, debemos hacer dos cosas:
 
 
    1. En nuestra view, pasamos objetos al template, para que pueda acceder datos, y generar el HTML dinámicamente.
        - mediante el diccionario "context"
        - rendereando usando el método render

In [ ]:
def atist_detail(request,artist_id): # le pasamos el artista que recibimos desde la API de modelos, usando el artist_id que recibe desde la API de la view.
    artist = Artist.objects.get(id=artist_id)
    context = {"artist": artist}
    return render(request, "music/artist_detail.html", context) #rendereamos el template. recibe por parámetros: el request, el template y el context.

    2. En el template, accedemos el context. 
    
    ¿Cómo?
    - Con llaves.
    - Las llaves dobles interpretan el código Python que hay dentro y lo imprimen:
        - Estás viendo el artista {{ artist.name }}

### Otras acciones...

##### Las llaves {% %} ejecutan el código Python sin imprimir

##### Control de flujo:

In [ ]:
{% for album in artist.album_set.all %}
    {{album.name}}
{% endfor %}

##### Agregar HTML

In [ ]:
<h1> Estás viendo al artista {{ artist.name}} </h1>
<ul>
{% for album in artist.album_set.all %}
    <li>{{album.name}}</li>
{% endfor %}
</ul>

##### Agregar CSS

- Creamos un directorio /static dentro de nuestra app.
- Dentro de {este, tenemos que creaor otro con el nombre de nuestra app. 
- Éste contendrá archivos estátivcos,como css y javascript.
- Dentro, podemos poner nuestras hojas de estilo, por ejemplo sytle.css

    Para referenciarlo en un template, ponemos lo siguiente:

In [ ]:
{% load static %}
<link rel="stylesheet" type="text/css", href="{% static "music/style.css" %}">

##### Herencia de Templates

- Nos permite reutilizar templates.
- Podemos tener una estructura para el HTML de nuestra aplicación.
- Podemos centralizar cosas como la importación de CSS y Javascript, o el layout general.


### 7. Forms

Hasta ahora, hemos visto aplicaciones estáticas y otras que despliegan HTML dinámicamente, desde un modelo de datos, usando una base de datos.

- Permiten a los usuarios interactuar con la plataforma web.

- ¿Cómo?
    - El protocolo HTTP tiene la capacidad para este caso de uso:
        - Métodos: POST, PUT, PATCH, DELETE.
  

  
### Elementos HTML que permiten recibir del usuario y enviarlo al sevidor mediante esto métodos HTTP.

Form

- Se usa para mostrar un formulario:

In [ ]:
<body>
    <form action="/someaction" method="post"> # action indica la ruta donde se enviarán los datos inggresado en el formulario. # method indica el método HTTP que se usará para enciar la información al servidor (valores posibles: get,post).
        ...
    </form>
</body>



- al enviar un formulario con **get**, los datos se enían como parte de la URL.
- en un formulario con **post**, los datos se envían como parte del cuerpo del request, no son visibles en la URL.

##### Elementos Input:

- para que el usuario pueda ingresar datos, se usan elementos input:



                        

In [ ]:
<body>
    <form action="/someaction" method="post">
        <input type="text" name="first_name"/> # el atributo type indica el tipo de input (texto, selección, archivo...)
        <input type="text" name="last_name"/> # el atributo name indica el nombe con el cual le llegará el valor ingresado por el usuario al servidor. Si no se incluye, el dato no será enviado.
    <form/>
</body>


##### Labels:

- texto explicativo para cada input.
- importantes para accesibilidad y usabilidad.

- el atributo for indica a cuál input corresponde el label a través del id del input.

In [ ]:
<body>
    <form action="/someaction" method="post">
        <label for="first-name">First name:<first_name/>
    </form>
</body>

##### Tipos de input

text
- se usa para ingresa un texto de una línea.

In [ ]:
<label for="first-name">First name:</label>
<input type"text" id="first-name" name="first_name"/>

radio
- permiten al usuario seleccionar una opción

In [ ]:
<input type "radio" id="input-green" name="color" value="green"/> #todas tienen el mismo name, para que sean excluyentes.
<label for="input-green">Verde</label>
<input type="radio" id="input-yellow" name="color" value="yellow"/>
<label for="input-yellow">Amarillo</label>
<input type="radio" id="input-red" name="color" value="red"/>
<label for="input-red">Rojo</label>

In [ ]:
checkbox

- permiten seleccionar multiples opciones

In [ ]:
<input type "checkbox" id="input-green" name="color_green" value="green"/> #todas tienen el mismo name, para que sean excluyentes.
<label for="input-green">Verde</label>
<input type="checkbox" id="input-yellow" name="color_yellow" value="yellow"/>
<label for="input-yellow">Amarillo</label>
<input type="checkbox" id="input-red" name="color_red" value="red"/>
<label for="input-red">Rojo</label>

In [ ]:
number

- permite al usuario ingresar un número.
- el browser impide que se ingreseb caracteres no numéricos.
- se pueden establecer límites que el browser validará al enviar el formulario.

In [ ]:
<label for="input_age">Edad</label>
<input type="number" id="input-age" name="age" min"1" max="99">

file

- permite al usuario enviar un archivo
- el atributo accept permite restringir el tipo de archivos


In [3]:
<label for="input-profile">Foto de Perfil</label>
<input type="file" id="input-profile" name="profile" accept="image/png,image/jpeg">

SyntaxError: invalid syntax (<ipython-input-3-18e5fd4dba80>, line 1)

select

- permite elegir una opción desde una lista desplegable

In [2]:
<label for="select-city">Selecciona una comuna:</label>
<select id="select-city" name="city">
    <option value="pirque">Pirque</option>
    <option value="santiago" selected>Santiago</option>
    <option value="talagante">Talagante</option>
</select>


SyntaxError: invalid syntax (<ipython-input-2-e95a26eb9fe3>, line 1)

enviar un form

- se usa un input de tipo submit
- el atributo value indica el texto que se mostrará en el botón.

<input type="submit" value="Enviar"/>

In [ ]:
### Form en Django

- se incorpora un formen el template.

- recordemos que en un elemento de tipo form, usamos el atributo action para indicar dónde irá el request.
    - en django podemos usar el helper URL para esto.
    

In [2]:
<h1>Artista</h1>
<form action="{% url "artist_detail" artist.id %}" method="post">
    {% csrf_token %}
    {{form}}
        <input type="text" name="name" value="{{artist.name}}"/>
        <input type="submit" value="Guardar">
   </form> 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [1]:
...

Ellipsis

### 8. Admins

- Django provee una interfaz de administración autogenerada a partir de los modelos.
- Se puede utilizar como backoffice o para un usuario administrador de la aplicación.
- Se configura mediante la clase ModelAdmin.

In [ ]:
Sistema de administración que permite uns sisitema de backoffice para los administradores.

- Django trae un sistema de usuarios y autenticación.

In [ ]:
# Para crear un usuario administrador en el comando:

python manage.py createsuperuser

python manage.py runserver # para ver sus efectos

##### Visualización de modelos
- Para ver los modelos de nuestra app debemos registrarlos en el archivo admin.py

In [ ]:
from django.contrib import admin

from .modelos import Album

admin.site.register(Album) # aplicamos un método de su clase, register, el cual recibe com oargumento (Album)

In [ ]:
##### Configuración

Por defecto, al registrar un modelo en el admin se agregan todo sus campos. Podemos configurarlo mediante la clase Modeladmin.
    -Campo exclude
    -Campo fields

### 9. Routing

- Mediante archivos conociedos como URLConfs podemos configurar el routing de nuestro proyecto django.

- Las URLConfs permiten generar nuestras URLs con gra flexibilidad, permitiendo incluir otras URLConfs o generar la configuración dinámicamente.

- Django también provee mecanismos para hacer url reversing, generando las URLs para nuestras views sin tener que ponerlas en duro en nuestro código.

##### URLConfs
- Son módulos Python para configurar URLs de un proyecto django.
- Mapean URLs a views.
- Al ser código python, se pueden referenciar otras URLCOnfs y se pueden construit de manera dinámica.

Ejemplo: 

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path("", views.index, name="index"),
]

In [ ]:
### Capturar valores
- Podemos usar <> para capturar valores desde la URL, que serán pasados a la view.
- El formato es <converter.name>, donde el converter es un tipo de dato opcional que convertirá lo que se ingrese al tipo Python correspondiente y name es el nombre con que se pasará a la view.
- Converters posibles: str, int, slug(cualquier texto), uuid, path.

from django.urls import path
from . import views

urlpatterns = [
    path("",views.index, name="index"),
    path(<"artist/<int:artist_id>/", views.artist_detail, name="artist_detail"),
]
        

##### Orden de las URLs

-Django procesa las URLs en orden.
- La ruta /artists71/ invocará la view artist_special, pues aparece primero, a pesar de que la última ruta también hace match.

In [ ]:
from django.urls import path
from . import views

##### Includes

Una URLConf puede incluir otras.

En el ejemplo, la URLConf del proyecto incluye la URLConf de la app music.

- Aquí cualquier ruta que comience con /music, django le sacará esa parte, y pasará el resto a la URLConf que incluimos para qu ela preocese.

- De esta manera, la url /music/artist/2/ invocará la view artist_detail con artist_id=2.

In [ ]:
from django.urls import path, incliude

urlpatterns = [
    path("music", include("music.urls"))
]

##### Usando URLs en views y templates

- Hemos visto cómo llegar desde una URL introducida por un usuario a la vista correspondiente (posiblemente con parámetros extraídos de la misma URL).

- El proceso inverso a veces es necesario, y es conocido como URL reversing.

- Django provee maneras de hacerlo tanto en views como en templates.

- Esto es muy importante puesto que es una mala práctica porner URLs "en duro" en nuestro código.

- En nuestra view podemos usar el método reverse:
    

In [ ]:
from django.http import HttpResponseRedirect
from django.urls import reverse

def artist_redirect(request, artist_id):
    return HttpResponseRedirect(
    reverse("artist_detail", kwargs={"artist_id":artist_id}))

# aquí estamos obteniendo la URL con nombre artist_detail y con el parámetro artist_id.
    # Por ejemplo, si artist_id es 2, obtenemos /music/artists/2/

##### Usando URLs en templates
En nuestros templates podemos usar el helper url:


In [ ]:
<a href= "{%% url "artist_detail" artis.id %}">Artist {{ artist.id}}</a>

# Aquí estamos obteniendo la URL con nombre artist_detail y con el parámetro artist.id.
# Por ejemplo, si artist.id es 2, obtenemos /music/artists/2/

### 10. Django Shell

In [ ]:
El comando shell inicia el interprete de django que nos permite interactuar con la API programática de django.

### 11. Javascript

- JavaScript es un lenguaje de programación utilizado, principalemente, en aplicaciones web para procesamiento en el lado del cliente.

- También se usa en otros contextos, como servidores Web.

- En el cliente, sus principales usos son manipulación de HTML/CSS, reaccionar a acciones del usuario, y realizar requests de manera asíncrona.

- La sintáxis de JavaScript incluye elementos tales como control de flujo, arreglos, objetos y funciones.



- Principal Herramienta de procesamiento del lado del cliente.

- Tiene como propósito dar vida a las páginas web.

Permite:
    - 1. Manipular HTML y CSS de una página
        - Insertar o borrar elementos, cambiar contenido, cambiar atributos, modificar CSS..
        
    - 2. Reaccionar a acciones del usuario
        - Envío de formularios, clicks, movimientos del mouse, teclado..
   
   - 3. Enviar requests al servidor sin recargar la página (AJAX, Fetch API)
   
   

In [ ]:
##### Variables

- JavaScript no tiene tipos estáticos, por lo que las variables se declaran sin tipo.
- Uso de variables numéricas y strings:
    

In [ ]:
# // cosnt es para declarar variables que no se re-asignarán
const a = 12;

# // let permite re-asignar la variable
let b = "hola";

b = "chao";

##### Condiciones y comparaciones

In [ ]:
# Igualdad
a === 12;
a !== 15;

# Comparaciones

a <= 3;
a > 0;

# Operaciones Lógicas

a >= 3 && a < 10;
a === "hola" 

In [ ]:
##### if (a !==12) {
//...
} else if (a<=3) {
    //...
} else {
    //...
}

while (a< 10) {
    //...
}

for (let i = 2; i++) {
    //...
}

In [ ]:
### Arreglos

# Declaración

const arr = [];

# // Agregamos elementos:
arr.push(1);
arr.push(2);
arr.push(3);

let sum = 0;
# // Iteración
for (const elem of arr) {
    sum += elem;
}
#// Acceso
arr[1];

#// Larfo
arr.length;


##### Objetos

En javascript se le denomina objetos a diccionarios llave-valor.


In [ ]:
# // Declaración

const obj = {
    key1: "value1",
    key2: 30
};

# // Acceso a propiedades del objeto
const v1 = obj.key;

# // Modificación de propiedades del objeto ["key1", "key2"]
const keys = Object.keys(obj);

# // Areglo con los valores del objeto ["value1", 25]
const values = Object.values(obj)

# // Arreglo con los pared del objeto, cada uno en un arreglo [["key1", "value1"],["key2","value2"]]
const values = Object.values(obj);

# // Arreglo con los pares del objeto, cada uno en un arreglo [["key1", "value1"]], ["key2", 30]]
const entries = Object.entries(obj);


##### Funciones

In [ ]:
function fibonacci(k) { # // declaración de una función
    if (k < 2) {
        return 1; # // Retorno de una función
    }
    let n0 = 1;
    let n1 = 1;
    for (let i =2; i <=k, i++) {
        const n2 = n0 +n1;
        n0 = n1;
        n1 = n2;
    }
    retunr n1;
}

fibonacci(5); # // Llamando a una función.

### 12. Manipulación del DOM y eventos

##### DOM

- El DOM (Document Object Model) es una representación de GTML de una página mediante objetos JavaScript que puede modificarse:
    - El punto de entrada es el objeto document.
        - Por ejemplo, document.body nos entrega el body.
    - Podemos modificar su contenido y estilo.

In [ ]:
##### Métodos de acceso

Existen multiples maneras de acceder a los distintos elementos a parte del body.



In [ ]:
# 1 
document.getElementById(id)
# Retorna el elemento que tenga el id dado, que debe ser único.

# 2
elem.querySelectorAll(selector)
# Retrona un arreglo con todos los elementos que estén dentro de elem que cumplan con el selector CSS dado.


##### Métodos de acceso

In [ ]:
# retorna el elemento con id header:
const header = document.getElementById("header")

# retorna todos los div dentro del body:
const divs = document.body.querySelectorAll("div")

# retorna todos los elementos con clase class dentro del primer div d la query anterior:
divs[0].querySelectorAll("class")